In [1]:
# nltk.download()
import pandas as pd
import numpy as np
import operator
import re
import nltk
import sys
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

# Extract Features From the Test.csv

In [2]:
Test = pd.read_csv('./Final_Test_Set.csv')

In [3]:
# Start of the Test_Set
Test.iloc[[10020]]

,answer_content,answer_id,answer_voted,question_id,user_id,user_agrees,user_edu,user_exp,user_followed,user_helped,user_interest,user_intro,user_recommends,user_saved,user_specialty,user_thanks,question_tags,question_content
10020,yes a replaced knee is often more swollen than...,1433245,1,1000543,11168524,189,uc san francisco school of medicine ca,24years,"[11193858, 11828102, 12242507, 11202045, 11139...","177,202",total hip replacement total knee replacement ...,i did my med school and residency at ucsf and ...,dr dearborn is an amazing doctor #nationaldoct...,NaN,orthopedic reconstructive surgery,111,nsaids surgery knee replacement,will swelling after knee replacement surgery g...


In [4]:
def extract_normal_features(path_in, path_out): 
    m = pd.read_csv(path_in)

    # calculate the features
    # question_id :
    question_id = m['question_id']
    question_id.to_csv(path_out+'/qid.txt')

    # vote :
    vote = m['answer_voted']
    vote.to_csv(path_out+'/vote.txt')

    # feature1 : length of the answer_content 
    #answer_content = m[['answer_content','user_id']]
    answer_content = m['answer_content']
    # f1 : length of the answer_content
    f1 = answer_content.str.len()
    f1.to_csv(path_out+'/f1.txt')
    #print answer_content


    # feature2 : length of the user_introduction
    user_intro = m['user_intro']
    #print user_intro
    f2 = user_intro.str.len()
    f2.to_csv(path_out+'/f2.txt')
    #print f2


    # feature3 : length of the question_content
    question_content = m['question_content']
    f3 = question_content.str.len()
    #print f3
    f3.to_csv(path_out+'/f3.txt')


    # feature4: user's experience years
    user_exp = m['user_exp']
    f4 = m['user_exp']
    f4 = f4.str.extract('([0-9]*)')
    f4.to_csv(path_out+'/f4.txt')
    #print f4

    # feature5 : length of the user's edu
    user_edu = m['user_edu']
    f5 = user_edu.str.len()
    f5.to_csv(path_out+'/f5.txt')
    #print f5


    # feature6 : number of topics of user's interest
    user_interest = m['user_interest']
    f6 = user_interest.str.len()
    f6.to_csv(path_out+'/f6.txt')
    #print f6


    # feature7 : number of the people user followed
    user_followed = m['user_followed']
    f7 = user_followed.str.len()
    f7.to_csv(path_out+'/f7.txt')
    #print f7

    # feature8 : number of the question tags
    question_tags = m['question_tags']
    f8 = question_tags.str.len()
    f8.to_csv(path_out+'/f8.txt')
    #print f8


    # feature9 : number of saved lives
    saved_lives = m['user_saved']
    f9 = saved_lives
    f9.to_csv(path_out+'/f9.txt')
    #print f9


    # feature10 : number of recieved thanks
    recieved_thanks = m['user_thanks']
    #recieved_thanks = str(recieved_thanks)
    f10 = recieved_thanks.str.replace(',','')
    f10.to_csv(path_out+'/f10.txt')
    #print f10


    # feature11 : number of agrees
    recieved_agrees = m['user_agrees']
    f11 = recieved_agrees.str.replace(',','')
    f11.to_csv(path_out+'/f11.txt')
    #print f11


    # feature12 : nuber of helped people
    helped = m['user_helped']
    f12 = helped.str.replace(',','')
    f12.to_csv(path_out+'/f12.txt')
    #print f12


    # feature13 : number of doctor recommend
    recommend = m['user_recommends']
    f13 = recommend.str.len()
    f13.to_csv(path_out+'/f13.txt')
    #print f13

    # feature14 : total length of the recommends
    # recommend_str = m['user_recommends']
    # recommend_str = ','.join(str(v) for v in recommend_str)
    # f14 = recommend_str.to_string
    # print f14
    #f14.to_csv('f14.txt')


# Extracting for the TEXT FEATRUES
def extract_text_features(path_in, path_lda):
    
    m = pd.read_csv(path_in)

    # Process with the question_content
    qc = m['question_content']
    #qc = str_strip(qc)
    #print qc
    qc.to_csv(path_lda+'/question_content.txt', encoding='utf-8')

    # Process with the answer_content
    ac = m['answer_content']
    #ac = str_strip(ac)
    ac.to_csv(path_lda+'/answer_content.txt', encoding='utf-8')

    # Process with the user_recommends
    ur = m['user_recommends'].astype(str)
    #ur = str_strip(ur)
    ur.to_csv(path_lda+'/user_recommends.txt', encoding='utf-8')

    # Process with question_tags
    qt = m['question_tags'].astype(str)
    #qt = str_strip(qt)
    #print qt
    qt.to_csv(path_lda+'/question_tag.txt', encoding='utf-8')

    # Process with user_interest
    ut = m['user_interest'].astype(str)
    #ut = str_strip(ut)
    ut.to_csv(path_lda+'/user_interest.txt', encoding='utf-8')

    # Proecess with user_specialty
    us = m['user_specialty'].astype(str)
    #us = str_strip(us)
    us.to_csv(path_lda+'/user_specialty.txt', encoding='utf-8')

In [5]:
def remove_stop_words(fin, fout) :
    for line in fin:
        word_list = line.split(',')[1].strip().split(' ')
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        print filtered_words
        fout.write(' '.join(filtered_words))
        fout.write('\n')

# do text - preprocessing 
# get tokens / remove stopwords / do stemming 
def text_processing(FIN, FOUT) :
    for line in FIN :
        stemmed = []

        # do some strip to get tokens 
        word_str = line.split(',')[1]
        word_str = word_str.replace('u\'', '\'')
        word_str = word_str.replace("'", "")
        word_str = re.sub(r"\W+", " ", word_str)
        word_list = word_str.strip().split(' ')
        
        # remove the stop words from tokens 
        filtered = [word for word in word_list if word not in stopwords.words('english')]
        # print filtered
        # do stemming on the word 
        # for word in filtered :
            # stemmed.append(PorterStemmer().stem(word))
            # stemmed.append(SnowballStemmer("english").stem(word))
        #FOUT.write(' '.join(stemmed))
        FOUT.write(' '.join(filtered))
        FOUT.write('\n')

    return 

def process_text_features(PATH_IN, PATH_OUT) :
    
    reload(sys)
    sys.setdefaultencoding('utf8')
    try :
        with open(PATH_IN+'question_content.txt','r') as f_in1,        \
            open(PATH_OUT+'question_content_pro.txt','w') as f_out1,   \
            open(PATH_IN+'answer_content.txt','r') as f_in2,           \
            open(PATH_OUT+'answer_content_pro.txt','w') as f_out2,     \
            open(PATH_IN+'user_interest.txt','r') as f_in3,            \
            open(PATH_OUT+'user_interest_pro.txt','w') as f_out3,      \
            open(PATH_IN+'question_tag.txt','r') as f_in4,             \
            open(PATH_OUT+'question_tag_pro.txt','w') as f_out4,       \
            open(PATH_IN+'user_recommends.txt','r') as f_in5,          \
            open(PATH_OUT+'user_recommends_pro.txt', 'w') as f_out5,   \
            open(PATH_IN+'user_specialty.txt', 'r') as f_in6,          \
            open(PATH_OUT+'user_specialty_pro.txt', 'w') as f_out6:    

                text_processing(f_in1, f_out1)
                text_processing(f_in2, f_out2)
                text_processing(f_in3, f_out3)
                text_processing(f_in4, f_out4)
                text_processing(f_in5, f_out5)
                text_processing(f_in6, f_out6)
    except IOError as e:
        print 'Operation failed: %s' % e.strerror

In [6]:
from itertools import izip

def unique(a):
    return list(set(a))

def intersect(a,b):
    return list(set(a)&set(b))

def union(a,b):
    return list(set(a) | set(b))

# Run this code for best_answer_features
# PATH_IN = 'DIVIDE/match/best_answer_features/OVER_LAP/'
# PATH_OUT = 'DIVIDE/match/best_answer_features/OVER_LAP/'
# PATH_FEATURES = 'DIVIDE/match/best_answer_features/'

def extract_overlap_features(PATH_IN, PATH_OUT, PATH_FEATURES) :
    fin_1 = open(PATH_IN + 'answer_content_pro.txt','r')
    fin_2 = open(PATH_IN + 'question_content_pro.txt','r')
    fin_3 = open(PATH_IN + 'user_interest_pro.txt')

    fout_1 = open(PATH_OUT + 'q_a_overlap.txt','w')
    fout_2 = open(PATH_OUT + 'q_u_overlap.txt','w')
    fout_3 = open(PATH_OUT + 'a_u_overlap.txt','w')

    f15 = open(PATH_FEATURES + 'f15.txt', 'w')
    f16 = open(PATH_FEATURES + 'f16.txt', 'w')
    f17 = open(PATH_FEATURES + 'f17.txt', 'w')

    count = 0

    for line1, line2, line3 in izip(fin_1, fin_2, fin_3):
        word_list1 = line1.strip().split(' ')
        word_list2 = line2.strip().split(' ')
        word_list3 = line3.strip().split(' ')

        over_lap1 = intersect(word_list1, word_list2)
        over_lap2 = intersect(word_list1, word_list3)
        over_lap3 = intersect(word_list2, word_list3)

        len1 = len(over_lap1)
        len2 = len(over_lap2)
        len3 = len(over_lap3)

        print over_lap1
        fout_1.write(str(count)+',')
        fout_1.write(' '.join(over_lap1))
        fout_1.write(','+str(len1)+'\n')

        fout_2.write(str(count)+',')
        fout_2.write(' '.join(over_lap2))
        fout_2.write(','+str(len2)+'\n')

        fout_3.write(str(count)+',')
        fout_3.write(' '.join(over_lap3))
        fout_3.write(','+str(len3)+'\n')


        f15.write(str(count)+','+str(len1)+'\n')
        f16.write(str(count)+','+str(len2)+'\n')
        f17.write(str(count)+','+str(len3)+'\n')

        count = count + 1

    fin_1.close()
    fin_2.close()
    fin_3.close()

    fout_1.close()
    fout_2.close()
    fout_3.close()

    f15.close()
    f16.close()
    f17.close()

In [7]:
def lda_source(PATH_IN, PATH_OUT) :

    f1 = open(PATH_IN + 'question_content_pro.txt','r')
    f2 = open(PATH_IN + 'answer_content_pro.txt','r')
    f3 = open(PATH_IN + 'user_interest_pro.txt', 'r')
    f4 = open(PATH_IN + 'question_tag_pro.txt', 'r')

    fout = open(PATH_OUT + 'TEST_SOURCE.txt','w')

    for line1, line2, line3, line4 in izip(f1, f2, f3, f4) :
        #line1 = f1.readline()
        info1 = line1.strip()
        print info1
        fout.write(info1)

        #line2 = f2.readline()
        info2 = line2.strip()
        print info2
        fout.write(' '+info2)

        info3 = line3.strip()
        print info3
        fout.write(' '+info3)

        info4 = line4.strip()
        print info4
        fout.write(' '+info4 + '\n')

    f1.close()
    f2.close()
    f3.close()
    f4.close()
    fout.close()

In [8]:
def normalize(PATH_IN, PATH_OUT) :
    # feature1 : length of the answer_content 
    f1 = pd.read_csv(PATH_IN + 'f1.txt', names = ['index','len_answer_content'])

    # feature2 : length of the user_introduction
    f2 = pd.read_csv(PATH_IN + 'f2.txt', names = ['index','len_user_intro'])
    m = pd.merge(f1, f2, how='outer')
    #print m

    # feature3 : length of the question_content
    f3 = pd.read_csv(PATH_IN + 'f3.txt', names = ['index', 'len_question_content'])
    m = pd.merge(m, f3, how='outer')

    # feature4: user's experience years
    f4 = pd.read_csv(PATH_IN + 'f4.txt', names = ['index','user_exp'])
    m = pd.merge(m, f4, how='outer')
    #print m

    # feature5 : length of the user's edu
    f5 = pd.read_csv(PATH_IN + 'f5.txt', names = ['index','len_user_edu'])
    m = pd.merge(m, f5, how='outer')

    # feature6 : number of topics of user's interest
    f6 = pd.read_csv(PATH_IN + 'f6.txt', names = ['index','num_user_interst'])
    m = pd.merge(m, f6, how='outer')

    # feature7 : number of the people user followed
    f7 = pd.read_csv(PATH_IN + 'f7.txt', names = ['index','num_user_followed'])
    m = pd.merge(m, f7, how='outer')

    # feature8 : number of the question tags
    f8 = pd.read_csv(PATH_IN + 'f8.txt', names = ['index', 'num_question_tags'])
    m = pd.merge(m, f8, how='outer')

    # feature9 : number of saved lives
    f9 = pd.read_csv(PATH_IN + 'f9.txt', names = ['index','num_user_saved'])
    m = pd.merge(m, f9, how='outer')

    # feature10 : number of recieved thanks
    f10 = pd.read_csv(PATH_IN + 'f10.txt', names = ['index','num_user_thanks'])
    m = pd.merge(m, f10, how='outer')

    # feature11 : number of agrees
    f11 = pd.read_csv(PATH_IN + 'f11.txt', names = ['index','num_user_agrees'])
    m = pd.merge(m, f11, how='outer')

    # feature12 : nuber of helped people
    f12 = pd.read_csv(PATH_IN + 'f12.txt', names = ['index','num_user_helped'])
    m = pd.merge(m, f12, how='outer')

    # feature13 : number of doctor recommend
    f13 = pd.read_csv(PATH_IN + 'f13.txt', names = ['index','num_user_recommend'])
    m = pd.merge(m, f13, how='outer')

    #####################################################################
    # feature15 : number of answer_question overlap
    f15 = pd.read_csv(PATH_IN + 'f15.txt', names = ['index', 'answer_question_overlap'])
    m = pd.merge(m, f15, how='outer')


    # feature16 : number of question_user_interest overlap
    f16 = pd.read_csv(PATH_IN + 'f16.txt', names = ['index', 'question_uinterest_overlap'])
    m = pd.merge(m, f16, how= 'outer')

    # feature17 : number of user_interest_answer overlap
    f17 = pd.read_csv(PATH_IN + 'f17.txt', names = ['index', 'uinterest_answer_overlap'])
    m = pd.merge(m, f17, how= 'outer')

    #######################################################################
    Index = m.ix[:,0] 
    M = m.ix[:,1:]

    M = (M - M.min()) / (M.max() - M.min())
    M['index'] = Index


    # question_id :
    # qid = pd.read_csv(PATH_IN + 'qid.txt', names = ['index', 'question_id'])
    # M = pd.merge(M, qid, how = 'outer')

    # number_of_vote
    # vote = pd.read_csv(PATH_IN + 'vote.txt', names = ['index', 'vote'])
    # M = pd.merge(M, vote, how = 'outer')

    M = M.drop('index', axis = 1)
    M = M.fillna(0)
    # print M

    # M.to_csv(PATH_OUT + 'Normal_Features.txt',  index = False)
    M.to_csv(PATH_OUT + 'Total_Normal.txt',  index = False)
    
    '''
    m_norm = (m-m.mean()) / (m.max() - m.min())
    print m_norm

    print m.max()
    print m.min()
    print m.mean()
    #print m.std()
    '''

    #M_norm.to_csv('Normaliztion.txt',  index = False)

In [9]:
PATH_IN = './Final_Test_Set.csv'
PATH_NORMAL = './NORMAL/'
PATH_OVERLAP = './OVERLAP/'
PATH_TEXT = './TEXT/'
PATH_LDA = './LDA/'

In [10]:
extract_normal_features(PATH_IN, PATH_NORMAL)

In [11]:
extract_text_features(PATH_IN, PATH_TEXT)

In [12]:
process_text_features(PATH_TEXT, PATH_OVERLAP)

In [13]:
extract_overlap_features(PATH_OVERLAP, PATH_OVERLAP, PATH_NORMAL)

In [ ]:
lda_source(PATH_OVERLAP, PATH_LDA)

In [16]:
normalize(PATH_NORMAL, PATH_NORMAL)

In [17]:
'''
import os

data_path = './LDA/'
with open('./LDA/LDA_SOURCE.txt', 'w') as outfile:
    for fname in os.listdir(data_path):
        if fname[0] == '.':
            continue
        path = os.path.join(data_path,fname)
        #print path

        with open(path,'r') as infile :
            outfile.write(infile.read())
'''

"\nimport os\n\ndata_path = './LDA/'\nwith open('./LDA/LDA_SOURCE.txt', 'w') as outfile:\n    for fname in os.listdir(data_path):\n        if fname[0] == '.':\n            continue\n        path = os.path.join(data_path,fname)\n        #print path\n\n        with open(path,'r') as infile :\n            outfile.write(infile.read())\n"

In [2]:
# Format the S2V
s2v_list = []
for i in range(50) :
    col_name = 's2v_' + str(i) 
    s2v_list.append(col_name)
# print s2v_list

In [3]:
# !!! before this , you should remove the first row of the Test_Source.txt.vec
total_s2v = pd.read_table('./SENTENCE_VECTOR/TEST_SOURCE.txt.vec', sep = ' ', names = s2v_list)

In [4]:
# test_s2v !!! very important : Normalization
total_s2v = (total_s2v - total_s2v.min()) / (total_s2v.max() - total_s2v.min())

In [5]:
# total_s2v # 11292 rows × 50 columns

In [6]:
total_s2v.to_csv('./SENTENCE_VECTOR/Total_S2V.csv', encoding='utf-8', index = False)

In [7]:
# T_s2v
topic_list = []
for i in range(50) :
    col_name = 'topic_' + str(i) 
    topic_list.append(col_name)
print topic_list

['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', 'topic_20', 'topic_21', 'topic_22', 'topic_23', 'topic_24', 'topic_25', 'topic_26', 'topic_27', 'topic_28', 'topic_29', 'topic_30', 'topic_31', 'topic_32', 'topic_33', 'topic_34', 'topic_35', 'topic_36', 'topic_37', 'topic_38', 'topic_39', 'topic_40', 'topic_41', 'topic_42', 'topic_43', 'topic_44', 'topic_45', 'topic_46', 'topic_47', 'topic_48', 'topic_49']


In [8]:
test_topic = pd.read_table('./LDA/model-final.theta', sep = ' ', names = topic_list, index_col=False)

In [9]:
# test_topic !!! very important : Normalization
test_topic = (test_topic - test_topic.min()) / (test_topic.max() - test_topic.min())

In [10]:
# test_topic = test_topic[10020:]

In [11]:
# test_topic
test_topic.to_csv('./LDA/Total_Topic.csv', encoding='utf-8', index=False)

In [12]:
# T_topic = pd.read_csv('TEST/LDA/test_topic.csv')
# T_topic

Total_Normal = pd.read_csv('./NORMAL/Total_Normal.txt')
Total_S2V = pd.read_csv('./SENTENCE_VECTOR/Total_S2V.csv')
Total_Topic = pd.read_csv('./LDA/Total_Topic.csv')

In [13]:
# test_normal
# test_s2v
# test_topic

In [14]:
Total_Normal_S2V = pd.merge(Total_Normal, Total_S2V, left_index=True, right_index=True)
Total_Final = pd.merge(Total_Normal_S2V, Total_Topic, left_index=True, right_index=True)

In [15]:
# Total_Final # 11921 x 116 

In [16]:
Total_Final.to_csv('./Total_Final_Feature.csv', encoding='utf-8', index=False)

In [17]:
# Total_Final  # 11292 x 116 

In [18]:
test_len = [17, 15, 16, 14, 13, 10, 11, 16, 11, 15, 12, 12, 12, 12, 13, 12, 16, 15, 15, 11, 12, 14, 11, \
            15, 13, 10, 12, 13, 18, 15, 13, 12, 15, 13, 13, 13, 10, 14, 12, 16, 12, 14, 12, 14, 12, 14, \
            14, 12, 10, 14, 14, 12, 13, 12, 11, 14, 10, 13, 15, 16, 15, 12, 13, 14, 14, 13, 12, 11, 13, \
            11, 12, 13, 11, 10, 12, 13, 16, 12, 13, 12, 14, 13, 18, 15, 14, 12, 13, 13, 13, 11, 14, 12, \
            14, 10, 15, 17, 15, 15, 12, 14, 12, 11, 13, 14, 13, 11, 12, 11, 12, 17, 10, 10, 15, 14, 13, \
            10, 14, 13, 15, 15, 13, 14, 14, 14, 15, 17, 13, 16, 12, 14, 14, 13, 14, 12, 13, 13, 12, 10, \
            16, 11, 14, 13, 12, 13, 13, 12, 12, 15, 14, 14, 13, 12, 18, 14, 13, 11, 16, 17, 14, 13, 12, \
            11, 14, 13, 13, 10, 13, 12, 13, 14, 12, 11, 13, 15, 12, 10, 15, 12, 13, 13, 10, 14, 15, 12, \
            12, 14, 18, 14, 14, 12, 11, 12, 13, 14, 12, 13, 14, 15, 14, 12]

In [19]:
# len(test_len) 50 

In [20]:
start = end = 10020
for i in range( len(test_len) ) :
    end += test_len[i]
    print (start, end)
    test_i = Total_Final[start:end]
    test_i.to_csv('../MODEL/SVM_Data/TEST/' + 'test_' + str(i) + '.csv', encoding='utf-8', index = False)
    start += test_len[i]

(10020, 10037)
(10037, 10052)
(10052, 10068)
(10068, 10082)
(10082, 10095)
(10095, 10105)
(10105, 10116)
(10116, 10132)
(10132, 10143)
(10143, 10158)
(10158, 10170)
(10170, 10182)
(10182, 10194)
(10194, 10206)
(10206, 10219)
(10219, 10231)
(10231, 10247)
(10247, 10262)
(10262, 10277)
(10277, 10288)
(10288, 10300)
(10300, 10314)
(10314, 10325)
(10325, 10340)
(10340, 10353)
(10353, 10363)
(10363, 10375)
(10375, 10388)
(10388, 10406)
(10406, 10421)
(10421, 10434)
(10434, 10446)
(10446, 10461)
(10461, 10474)
(10474, 10487)
(10487, 10500)
(10500, 10510)
(10510, 10524)
(10524, 10536)
(10536, 10552)
(10552, 10564)
(10564, 10578)
(10578, 10590)
(10590, 10604)
(10604, 10616)
(10616, 10630)
(10630, 10644)
(10644, 10656)
(10656, 10666)
(10666, 10680)
(10680, 10694)
(10694, 10706)
(10706, 10719)
(10719, 10731)
(10731, 10742)
(10742, 10756)
(10756, 10766)
(10766, 10779)
(10779, 10794)
(10794, 10810)
(10810, 10825)
(10825, 10837)
(10837, 10850)
(10850, 10864)
(10864, 10878)
(10878, 10891)
(10891, 10